In [66]:
import os
import streamlit as st
import pickle 
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [69]:
llm = OpenAI(temperature=0.9, max_tokens=500)

In [70]:
loader = UnstructuredURLLoader(urls = [
    "https://www.moneycontrol.com/news/india/im-being-framed-why-kolkata-rape-murder-case-accused-broke-down-while-giving-consent-for-lie-detector-test-12805788.html"
     ,"https://timesofindia.indiatimes.com/business/india-business/unified-pension-scheme-approved-by-pm-modi-led-cabinet-check-salient-features/articleshow/112766588.cms"
])
data = loader.load()


# SPLITING AND MERGING

In [71]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

docs = text_splitter.split_documents(data)
len(docs)

26

In [74]:
embeddings = OpenAIEmbeddings()

vectorindex_openai = FAISS.from_documents(docs,embeddings)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
file_path = "vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex_openai, f)

In [ ]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

In [ ]:
query = " "

langchain.debug=True

chain({"questions": query}, return_only_outputs=True)